In [ ]:
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import csv
import time
import pandas as pd

# Read the CSV file containing certificate numbers
df = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\Sample - Data SCrapping_1 - RERA Website - 25.7.223.csv")
df = df.iloc[::,0]

# Initialize Edge WebDriver
driver = webdriver.Edge(EdgeChromiumDriverManager().install())

# Create CSV file with headers only
with open("Rera_website_scraping_sample_1.csv", 'a+', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    header = ["RERA Cert No", "Organisation Type", "Project Name", "Promoter Name", "Office Number", "Member Information", "Litigations related to the project ?", "Project Type", "Are there any Promoter(Land Owner/ Investor) (as defined by MahaRERA Order) in the project ?", "Plot Bearing No / CTS no / Survey Number/Final Plot no.", "Project Village", "Project Pin Code", "Project Plot Area", "Bank Name", "Bank IFSC", "Architect Name ", "Engineer Name", "Contractor Name", "Chartered Accountant Name", "Real Estate Agent Name", "Real Estate Agent Certificate No.", "No. Of Form 1 Uploaded", "No. of Form 2 uploaded", "website", "Total_Number_of_Proposed_Building_Wings", "Built_up_Area", "Sanctioned_FSI_of_the_project_applied_for_registration", "Permissible_Total_FSI_of_Plot"]
    writer.writerow(header)

# Loop through each certificate number
for i in df:
    print(i)
    view = 0
    driver.implicitly_wait(2)
    driver.get('https://maharerait.mahaonline.gov.in/searchlist/search?MenuID=1069')
    driver.find_element("xpath", "//input[@value ='Promoter']").click()
    project_input_box = driver.find_element("xpath", "//input[@id='CertiNo']")
    project_input_box.send_keys(i)
    driver.find_element("xpath", "//input[@id='btnSearch']").click()
    parent_handle = driver.current_window_handle
    
    try:
        # Try to click on the 'View' link to view project details
        driver.find_element("xpath", "//a[@target='_blank'][contains(text(),'View')]").click()
    except:
        try:
            # If the first attempt fails, try again after a delay
            driver.implicitly_wait(5)
            driver.find_element("xpath", "//a[@target='_blank'][contains(text(),'View')]").click()
        except:
            try:
                # If the second attempt fails, try one more time
                driver.implicitly_wait(5)
                driver.find_element("xpath", "//a[@target='_blank'][contains(text(),'View')]").click()
            except:
                # If all attempts fail, increment view counter and continue to the next certificate number
                view += 1
                pass
    
    # If view is successful, extract project details
    if view == 0:
        handles = driver.window_handles
        driver.switch_to.window(handles[1])
        project_name = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),'  Project Name')]//parent::div//following-sibling::div").text
        organization_type = driver.find_element("xpath", "//label[@for ='PersonalInfoModel_InfoTypeValue']//parent::div//following-sibling::div").text
        
        # Extracting details based on organization type
        if organization_type == "Other Than Individual":
            promoter_name = driver.find_element("xpath", "//label[@for ='PersonalInfoModel_CompanyName']//parent::div//following-sibling::div").text
            Member_info = driver.find_elements("xpath", "//div[@id='fldindtxt78']//table//tr[position() > 1]")
            office_number = driver.find_element("xpath", "//label[@for ='PersonalInfoModel_CompanyOfficeNo']//parent::div//following-sibling::div").text
            Member_info_list = []
            for j in Member_info:
                Member_info_list.append(j.find_element("xpath", ".//td[1]").text)
        else:
            Member_info_list = ["N/A"]
            office_number = driver.find_element("xpath", "//label[@for ='PersonalInfoModel_IndivisualOfficeNo']//parent::div//following-sibling::div").text
        
        litigation_related_to_this_project = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),'  Litigations related to the project ?')]//parent::div//following-sibling::div").text
        project_type = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),'Project Type')]//parent::div//following-sibling::div").text
        promoter_land = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),'Are there any Promoter(Land Owner/ Investor) (as defined by MahaRERA Order) in the project ?')]//parent::div//following-sibling::div").text
        plot_no = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),' Plot Bearing No / CTS no  / Survey Number/Final Plot no.')]//parent::div//following-sibling::div").text
        village = driver.find_element("xpath", "//div[@id='DivProject']//div[@class='x_content label-block']//div[@class='col-md-3 col-sm-3']/label[contains(text(),'Village')]//parent::div//following-sibling::div").text
        pin_code = driver.find_element("xpath", "//div[@id='DivProject']//div[@class='x_content label-block']//div[@class='form-group']//div[@class='col-md-3 col-sm-3']/label[contains(text(),'Pin Code')]//parent::div//following-sibling::div").text
        plot_area = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),'Total Plot/Project area')]//parent::div//following-sibling::div").text
        bank_name = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),'Bank Name')]//parent::div//following-sibling::div").text
        ifsc_code = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']/label[contains(text(),' IFSC Code')]//parent::div//following-sibling::div").text

        # Extracting architect details
        architect = driver.find_elements("xpath", "//td[contains(text(),'Architect')]//preceding-sibling::td[position() = 2]")
        archi = []
        for k in architect:
            archi.append(k.text)
        if len(archi) == 0:
            archi.append("N/A")

        # Extracting contractor details
        contractor = driver.find_elements("xpath", "//td[contains(text(),'Contractor')]//preceding-sibling::td[position() = 2]")
        contr = []
        for l in contractor:
            contr.append(l.text)
        if len(contr) == 0:
            contr.append("N/A")

        # Extracting engineer details
        engineer = driver.find_elements("xpath", "//td[contains(text(),'Engineer')]//preceding-sibling::td[position() = 2]")
        engi = []
        for m in engineer:
            engi.append(m.text)
        if len(engi) == 0:
            engi.append("N/A")

        # Extracting real estate agent details
        agent_name = driver.find_elements("xpath", "//td[contains(text(),'Real Estate Agent')]//preceding-sibling::td[position() = 2]")
        if len(agent_name) > 300:
            agent_name = agent_name[:300]
        agname = []
        for n in agent_name:
            agname.append(n.text)
        if len(agname) == 0:
            agname.append("N/A")

        # Extracting chartered accountant details
        ca = driver.find_elements("xpath", "//td[contains(text(),'Chartered Accountant')]//preceding-sibling::td[position() = 2]")
        char = []
        for o in ca:
            char.append(o.text)
        if len(char) == 0:
            char.append("N/A")

        # Extracting real estate agent certificate numbers
        agent_number = driver.find_elements("xpath", "//td[contains(text(),'Real Estate Agent')]//preceding-sibling::td[position() = 1]")
        if len(agent_number) > 300:
            agent_number = agent_number[:300]
        agent_num = []
        for p in agent_number:
            agent_num.append(p.text)
        if len(agent_num) == 0:
            agent_num.append("N/A")

        # Counting the number of Form 1 certificates uploaded
        form_1 = driver.find_elements("xpath", "//span[contains(text(),'Certificates of Architect (Form 1)')]")
        form_1_count = len(form_1)
        
        # Counting the number of Form 2 certificates uploaded
        form_2 = driver.find_elements("xpath", "//span[contains(text(),'Certificates of Engineer (Form 2)')]")
        form_2_count = len(form_2)

        # Extracting website
        website = driver.find_element("xpath", "//label[@for='PersonalInfoModel_WebsiteURL']//parent::div//following-sibling ::div").text
        Total_Number_of_Proposed_Building_Wings = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']//label[contains(text(),'Total Number of Proposed Building/Wings (In the Layout/Plot')]//parent :: div//following-sibling::div").text
        Built_up_Area = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']//label[contains(text(),'  Built-up-Area as per Proposed FSI (In sqmts) ( Proposed but not sanctioned) (As soon as approved, should be immediately updated in Approved FSI)')]//parent :: div//following-sibling::div").text
        Sanctioned_FSI_of_the_project_applied_for_registration = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']//label[contains(text(),'Sanctioned FSI of the project applied for registration  (Sanctioned Built-up Area)')]//parent :: div//following-sibling::div").text
        Permissible_Total_FSI_of_Plot = driver.find_element("xpath", "//div[@class='col-md-3 col-sm-3']//label[contains(text(),'  Permissible Total FSI of Plot (Permissible Built-up Area)')]//parent :: div//following-sibling::div").text
        driver.close()
        driver.switch_to.window(handles[0])
        
        # Updating promoter name if the organization type is Individual
        if organization_type != "Other Than Individual":
            promoter_name = driver.find_element("xpath", "//td[@data-name='Name']").text

        # Writing row to CSV file
        rows = [i, organization_type, project_name, promoter_name, office_number, Member_info_list, litigation_related_to_this_project, project_type, promoter_land, plot_no, village, pin_code, plot_area, bank_name, ifsc_code, archi, engi, contr, char, agname, agent_num, form_1_count, form_2_count, website, Total_Number_of_Proposed_Building_Wings, Built_up_Area, Sanctioned_FSI_of_the_project_applied_for_registration, Permissible_Total_FSI_of_Plot]
        with open("Rera_website_scraping_sample_1.csv", 'a+', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(rows)
    else:
        # If view failed, write only the certificate number to CSV file
        rows = [i]
        with open("Rera_website_scraping_sample_1.csv", 'a+', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(rows)
        pass
